Author: Prakrut Kansara, Kris Su

Description:

Date created: 


In [1]:
import xarray as xr 
import pandas as pd
import numpy as np
import os

forecast in the past will be hindcast,

few changes for the code, 

he only things will change is the time step: for each month in Jan 2023 up to october of 2024 hindcast going 6 months into the future. 

Servir project, and the dashboard, incorporate LDAS into 

## Functions

In [2]:
## Function to read and save forecast data ##
## Create function to read data and save it in RAM as xarray Dataset
def read_trim_forecast(file_path, va):
    """
    Args:
        file_path (str): directory of 
        va (str): variable we will be looking for 

    Return:
        forecast_data (xarray.core.dataset.Dataset): trimmed dataset
    """
    try:
        forecast_data = xr.open_dataset(file_path)[va]
        
        # if initialization_date != None:
           # forecast_data = forecast_data.sel(time=initialization_date)
        # else:
            #forecast_data = forecast_data.sel(time=-1)

        return forecast_data
    
    except KeyError:
        print(f'Variable {va} not found in dataset')

In [3]:
## Function to read and save hindcast data ##
## Create function to read data and save it as netcdf file ##
def read_trim_hindcast(file_path, va):
    """
    Args:
        file_path (str): file path to hindcast data
        va (str): variable we will be looking for in the datatset
    Returns:
        hindcast_data (xarray.core.dataset.Dataset):
    """

    try:
        hindcast_data = xr.open_mfdataset(file_path)[va].chunk(dict(time=-1))
        return hindcast_data
    except KeyError:
        print(f'Variable {va} not found in dataset')

In [4]:
## Function to calculate threshold for forecast categories defined by quantiles
def get_thresh(icat, quantiles, xrds, dims=['ensemble','time']):
    """
    Args:
        icat (int): the ith category, for looping through three categories
        quantiles (list): pre-defined boundaries for ranking
        xrds (xarray.core.dataset.Dataset): xarray dataset of forecast 
        dims (list): get thresh for each variables for 
    
    Returns:
        xrds_lo (xarray.core.dataset.Dataset): lower threshold 
        xrds_hi (xarray.core.dataset.Dataset): higher threshold 
    """
    
    if not all(elem in xrds.dims for elem in dims): 
        raise Exception('Some of the dimensions in {} is not present in the xr.Dataset {}'.format(dims,xrds))

    else:
        if icat == 0: # category 0
            xrds_lo = -np.inf
            xrds_hi = xrds.quantile(quantiles[icat],dim=dims)
        elif icat == len(quantiles): # category MAX
            xrds_lo = xrds.quantile(quantiles[icat-1],dim=dims) 
            xrds_hi = np.inf # the upper bound extent to positive infinite
        else:
            xrds_lo = xrds.quantile(quantiles[icat-1],dim=dims)
            xrds_hi = xrds.quantile(quantiles[icat],dim=dims)
    
    return xrds_lo, xrds_hi

In [5]:
## Function to calculate probabilities for tercile categories by counting members within each category
def calculate_probabilities(hcst, fcst, quantiles = [1/3., 2/3.]):
    """
    Args:
        hcst (xarray.core.dataset.Dataset): hindcast data
        fcst (xarray.core.dataset.Dataset): forecast data
        quantiles (list): pre-defined boundaries for ranking

    Returns:
        probs (xarray.core.dataset.Dataset): 

    """
    print('Computing probabilities...')
    numcategories = len(quantiles)+1 # there are 3 categories in total

    # Mask out 0 values in forecast (set to nan)
    fcst_masked = fcst.where(fcst != 0)

    l_probs=list() # initialize empty list
    for icat in range(numcategories): # for each category
        print(f'category={icat}')
        h_lo, h_hi = get_thresh(icat, quantiles, hcst)
        prob = np.logical_and(fcst_masked>h_lo, fcst_masked<=h_hi).sum('ensemble')/float(fcst_masked.sizes['ensemble'])
        # Instead of using the coordinate 'quantile' coming from the hindcast xr.Dataset
        # we will create a new coordinate called 'category'
        #print(prob)

        #print(prob.coords)

        if 'quantile' in prob.coords:
            prob = prob.drop_vars('quantile')
        
        l_probs.append(prob.assign_coords({'category':icat}))
    
    #print(l_probs)
    probs = xr.concat(l_probs,dim='category')
    # print(f'Saving {aggr} tercile probs netCDF files')
    # probs.to_netcdf(f'{DATADIR}/{hcst_bname}.{aggr}.tercile_probs.nc')

    return probs

In [6]:
## Function to calculate two-category probabilities for CEMADEN ##
## Only save the data for above normal category air temperature and below normal precipitation ##

"""def calculate_two_ways_probabilities(hindcast, forecast, model, variable, initialization_date):
    cemaden_probs = calculate_probabilities(hindcast, forecast, quantiles=[1/2.])*100

    if variable=='prec':
        cemaden_probability = xr.Dataset(data_vars = {'below_normal':(['L','Y','X'],cemaden_probs.sel(category=0)[variable].data)}, 
                                            coords=dict(L=forecast.L.values,Y=forecast.Y.values, X=forecast.X.values), 
                                            attrs=forecast.attrs)
    elif variable=='tref':
        cemaden_probability = xr.Dataset(data_vars = {'above_normal':(['L','Y','X'],cemaden_probs.sel(category=1)[variable].data)}, 
                                            coords=dict(L=forecast.L.values,Y=forecast.Y.values, X=forecast.X.values), 
                                            attrs=forecast.attrs)
        
    ## Save the forecast data to a netcdf file ##
    year = forecast.time.dt.year.values
    month = forecast.time.dt.month.values
    day = forecast.time.dt.day.values

    cemaden_probability.to_netcdf(output_directory + initialization_date.replace('-','_') + '/cemaden_two_probability_' + model + '_' + variable + '_' + str(year) + '_' + str(month).zfill(2) + '_' + str(day).zfill(2) + '.nc')

    print('CEMADEN: Two probability categories calculated and saved for NMME hindcast data for ' + model + ' for ' + variable + '!')"""

"def calculate_two_ways_probabilities(hindcast, forecast, model, variable, initialization_date):\n    cemaden_probs = calculate_probabilities(hindcast, forecast, quantiles=[1/2.])*100\n\n    if variable=='prec':\n        cemaden_probability = xr.Dataset(data_vars = {'below_normal':(['L','Y','X'],cemaden_probs.sel(category=0)[variable].data)}, \n                                            coords=dict(L=forecast.L.values,Y=forecast.Y.values, X=forecast.X.values), \n                                            attrs=forecast.attrs)\n    elif variable=='tref':\n        cemaden_probability = xr.Dataset(data_vars = {'above_normal':(['L','Y','X'],cemaden_probs.sel(category=1)[variable].data)}, \n                                            coords=dict(L=forecast.L.values,Y=forecast.Y.values, X=forecast.X.values), \n                                            attrs=forecast.attrs)\n        \n    ## Save the forecast data to a netcdf file ##\n    year = forecast.time.dt.year.values\n    month = f

## Executable

In [7]:
workspace_directory = rf'C:\Users\Kris\Documents\amazonforecast\data\hindcast'
surface_model_file_path = workspace_directory + '/amazon_forecast/'
forecast_data_file_path = surface_model_file_path + "/ldas_fcst_2024_dec01.nc"

year = 2024
month = 'dec'

hindcast_data_file_path = []
for i in range(2001, year):
    hindcast_data_file_path.append(surface_model_file_path + os.path.join(f"/ldas_fcst_{i}_{month}01.nc"))
#hindcast_data_file_path

In [11]:
## Global variables
surface_variable_short = ['Rainf_tavg', 'Qair_f_tavg',
                          'Qs_tavg','Evap_tavg', 'Tair_f_tavg',
                          'SoilMoist_inst', 'SoilTemp_inst']

surface_variable_long = ['precipitation rate', 'specific humidity', 'surface runoff',
                         'total evapotranspiration', 'avg air temperature', 'soil moisture content', 'SoilTemp inst']

surface_variable_unit = {'Rainf_tavg':'mm/day', 'Qair_f_tavg':'g/kg', 'Qs_tavg':'mm/day',
                         'Evap_tavg':'mm/day', 'Tair_f_tavg':'degree Celsius', 'SoilMoist_inst':'m^3 m-3', 'SoilTemp_inst':'degree Celsius'}

In [15]:
for variable, variable_longname in zip(surface_variable_short, surface_variable_long):
    
    
    print(variable_longname)

    hindcast = read_trim_hindcast(hindcast_data_file_path, va=variable)

    initialization_date = '2024-12-31'
    forecast  = read_trim_forecast(forecast_data_file_path, variable)

    probs = calculate_probabilities(hindcast, forecast) *100

    max_prob = probs.max(dim='category')
    mask = probs != max_prob
    probs_with_nan = xr.where(mask, np.nan, probs)

    file_savepath = './get_ldas_probabilistics_output' + '/prob_' + initialization_date.replace('-','_')
    #create_directory(file_savepath)

    # 
    if variable in ['SoilMoist_inst', 'SoilTemp_inst']: 
        probs_with_nan.to_netcdf(file_savepath + '_tercile_probability_max_' + str(variable) + '.nc')
    else: 
        probs_with_nan.to_netcdf(file_savepath + '_tercile_probability_max_' + str(variable) + '_lvl_0' + '.nc')

    print('Saved probability values for ' + str(variable) + '!')


precipitation rate
Computing probabilities...
category=0
category=1
category=2
Saved probability values for Rainf_tavg!
specific humidity
Computing probabilities...
category=0
category=1
category=2
Saved probability values for Qair_f_tavg!
surface runoff
Computing probabilities...
category=0
category=1
category=2
Saved probability values for Qs_tavg!
total evapotranspiration
Computing probabilities...
category=0
category=1
category=2
Saved probability values for Evap_tavg!
avg air temperature
Computing probabilities...
category=0
category=1
category=2
Saved probability values for Tair_f_tavg!
soil moisture content
Computing probabilities...
category=0
category=1
category=2
Saved probability values for SoilMoist_inst!
SoilTemp inst
Computing probabilities...
category=0
category=1
category=2
Saved probability values for SoilTemp_inst!
